**The objective of this challenge is to forecast the number of turtles caught per week per capture site. This will help Local Ocean plan their staff schedules and budget.**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 200, "display.max_rows", 200)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [ ]:
turtle = pd.read_csv("train.csv", delimiter=",")
varDesc = pd.read_csv("variable_definitions.csv", engine="python")
submissionSample = pd.read_csv("Sample_sub.csv")
turtle.shape

initialRows = turtle.shape[0]

In [ ]:
turtle.head()

There are some identified inconsisenties: 
* For example, "Creek" and "creek" both mean the same thing. In this case, we will capitalize all entries within the said variable. <br>
* Converting the date variables to date time datatype. <br> 
* Drop the string row values in the Date_TimeRelease column. <br>

In [ ]:
turtle["ForagingGround"] = turtle["ForagingGround"].apply(lambda x: str(x).capitalize())
turtle["CaptureMethod"] = turtle["CaptureMethod"].apply(lambda x: str(x).capitalize())

In [ ]:
# releaseRows = ["ReleaseSite_50", "Released", "ReleaseSite_62", "ReleaseSite_56", "ReleaseSite_8", 
#                " green algae on the right inframarginal", " notch on fifth CS", "ReleaseSite_26", 
#                "ReleaseSite_21", " sm Bs all flips + shoulders", " barnacles (tiny) around neck", 
#                " notches on rear MS", "ReleaseSite_68", "ReleaseSite_19", "ReleaseSite_18", "ReleaseSite_11", 
#                "ReleaseSite_73", " notch on eleventh PMS", " tag hole on LRF", " tip of fifth CS missing", 
#                "ReleaseSite_9", " covered with thick green algae. Euthanised 27/9", 
#                " few small barnacles on head& FF + plastron& net damage on RFF", " flaking plastron", 
#                "ReleaseSite_39", " calcerous algal patches to rear", " sm bs on flips", 
#                " fresh wounds on RFs", " indent 7RMS", " a v-shaped notch in the middle of RFF", 
#                "ReleaseSite_35", " inside LFFsmall notch wound", "ReleaseSite_17", 
#                " LFflip inside last 3rd small fresh wound", " small piece on LRF missing", 
#                " calcareous algae on the plastron", " small healing tissue on neck", 
#                " outer edge of both RF are frayed", " few small barnacles on flippers", 
#                ' 1 b at junction of 1 CS+ 2 CS + 1 LLS', ' scar 3LLS', ' the edge of the LFF is jagged', 
#                ' swelling caused by the tagging', ' at least 50% of the right rear flipper is missing', 
#                ' 5 postoculars on the left side', ' 3 post-oculars on either side', 
#                ' 5 post-oculars on the right side',  ' 1 B on 2 CS& 11 RMS& R supra and LFF', 
#                "ReleaseSite_70", 'ReleaseSite_52', ' some flaking at scute joins', ' end supras broken off', 
#                ' 2 sm wounds on neck (scraping)', " calcareous algae on eigth CS", 
#                " flaking on edges of scutes", " small Bs on front flippers", " ", 
#                " flaking on the joints& light greenish algae on CS", "ReleaseSite_7", " light algae covering", 
#                " chips (inside edeg) and net damage on front flippers", " net damage on tip of LFF", 
#                " green and white calcareous algae on the plastron", " small dent on the nuchal scute", 
#                "ReleaseSite_4", "ReleaseSite_23", " some tiny barnacles on plastron", "ReleaseSite_60", 
#                " much red/green algae & bs on underside", "ReleaseSite_0", " RSCS missing tip", 
#                " small B on R supra", " light green algae on shell", " healing wound tissue at join 4CS/4RLS", 
#                " tag rip in LFF", " small barnacles on the carapace", 
#                " small barnacles on plastron and around the tail", " clean plastron", " flaking on plastron", 
#                " soft carapace", "ReleaseSite_32", 
#                "  notch in left supracandal + between supras.+ lost its tag from left rear flipper some time ago+ no fresh wound+ a little light algae covering", 
#                "ReleaseSite_29", " = all small bs", " light green algae on carapace", " supras half broken", 
#                " lge notch 10LMS& 8&10 and 11RMS", " tag rips on both RFs", " green algae on the plastrons", 
#                " barnacles on plastron and flippers (removed)", " 6CS", " L supracaudal is half broken", 
#                " scutepeeling on 10th and 11th RMS", " many sm bs on rear of shell& chip RRF", 
#                " b scar 1LLs & 5CS& sm chip RRF", " notch on join 10/11LMS", 
#                " right front flipper has b's at end", " flaking from 2nd>5th CS and 10th RMS", 
#                " a small scute between fourth and fifth LLS", "  a chip  9th LM", 
#                ' edgers of both RFs are jagged', ' few white patches of cancerous algae on rear scutes', 
#                "ReleaseSite_55", " 2/3 of LRF is missing", ' green algae on the plastron', 
#                ' thin green algae on both sides of the inframarginals', " green algae on rear carapace", 
#                " tag rip on LRF"
# ]
# turtle = turtle[~(turtle["Date_TimeRelease"].isin(releaseRows))]

# print("{} rows dropped. {} values are none date in date time release".format(initialRows - turtle.shape[0], 
#                                                                              len(releaseRows)))

In [ ]:
# convert to datetime datatype
turtle["Date_TimeCaught"] = pd.to_datetime(turtle["Date_TimeCaught"])
# turtle["Date_TimeRelease"] = pd.to_datetime(turtle["Date_TimeRelease"])
turtle[["Date_TimeCaught", "Date_TimeRelease"]].dtypes

In [ ]:
# separate the year and week from the date provided
turtle["Date_TimeCaughtYear"] = turtle["Date_TimeCaught"].apply(lambda x: x.strftime("%Y"))
turtle["Date_TimeCaughtWeek"] = turtle["Date_TimeCaught"].apply(lambda x: x.strftime("%W"))

# turtle["Date_TimeReleaseYear"] = turtle["Date_TimeRelease"].apply(lambda x: x.strftime("%Y"))
# turtle["Date_TimeReleaseWeek"] = turtle["Date_TimeRelease"].apply(lambda x: x.strftime("%W"))

turtle[["Date_TimeCaughtYear", "Date_TimeCaughtWeek"]].head()

It appears that most of the turtles are released on the same day they are captured. With this information, we will fill in the null released date for turtles with the released status similar to the date they were caught.

In [ ]:
turtle[turtle["Date_TimeRelease"].isna()]

In [ ]:
turtle["Date_TimeRelease"] = np.where((turtle["Date_TimeRelease"].isnull()) & (turtle["Status"] == "Released"), turtle["Date_TimeCaught"], turtle["Date_TimeRelease"])

In [ ]:
turtle["Status"] = np.where((turtle["Date_TimeRelease"].notnull()) & (turtle["Status"] != "Released"), "Released", turtle["Status"])

In [ ]:
turtle[turtle["Status"] == "Admitted"]["Date_TimeRelease"].unique()

All turtles that were admitted, do not have a release date. The assumption in this case would be that all admitted turtles are not released.
<br><br>
The Status column is very messy. For that reason, it shall be dropped.

In [ ]:
turtle[turtle["Date_TimeRelease"].isna()]["Status"].unique()

## Exploratory Data Analysis

In [ ]:
sns.countplot(y="ForagingGround", data=turtle)

In [ ]:
sns.countplot(y="CaptureMethod", data=turtle, order=turtle["CaptureMethod"].value_counts().index)

In [ ]:
sns.countplot(y="Status", data=turtle, order=turtle["Status"].value_counts().index)

In [ ]:
sns.distplot(turtle["CCL_cm"], kde=False)

In [ ]:
sns.distplot(turtle["CCW_cm"], kde=False)

In [ ]:
sns.distplot(turtle["Weight_Kg"], kde=False)

In [ ]:
turtle.isnull().sum()

In [ ]:
turtle.fillna(value={"Weight_Kg": 0, "Sex": "Unknown", "Lost_Tags": "None", "T_Number": "None"}, inplace=True)
turtle.replace({"Sex": {"Not_Recorded": "Unknown"}}, inplace=True)

In [ ]:
turtle["Sex"].unique()

In [ ]:
turtle["Weight_Kg"].head(2)

In [ ]:
submissionSample.head(2)

In [ ]:
turtle["Target"] = turtle["CaptureSite"] + "_" + turtle["Date_TimeCaughtYear"] + turtle["Date_TimeCaughtWeek"]
turtle[["Target", "CaptureSite"]].head()

In [ ]:
trainTurtle = turtle.groupby(["Target", "CaptureSite", "Date_TimeCaughtYear", "Date_TimeCaughtWeek"]).agg({"Rescue_ID": "count", "Researcher": "nunique", 
                                                             "LandingSite": "nunique", 
                                                             "ForagingGround": "nunique", 
                                                             "CaptureMethod": "nunique", "Fisher": "nunique", 
                                                             "Species": "nunique", "Lost_Tags": "nunique", 
                                                             "CCL_cm": "mean", "CCW_cm": "mean", 
                                                             "Weight_Kg": "mean", "Sex": "nunique", 
                                                             "Status": "nunique", 
                                                             "ReleaseSite": "nunique"}).reset_index()
trainTurtle.shape

In [ ]:
trainTurtle.rename(columns={"Target": "ID", "Rescue_ID": "Capture_Number"}, inplace=True)

In [ ]:
trainTurtle["Date_TimeCaughtWeek"] = trainTurtle["Date_TimeCaughtWeek"].astype(int)

In [ ]:
trainTurtle["Date_TimeCaughtYear"] = pd.to_numeric(trainTurtle["Date_TimeCaughtYear"])

In [ ]:
trainTurtle.head()

In [ ]:
trainTurtle.sort_values(by=["Capture_Number"], ascending=False).head()

In [ ]:
sns.scatterplot(x="Capture_Number", y="Researcher", data=trainTurtle)

In [ ]:
sns.scatterplot(x="Capture_Number", y="CCL_cm", data=trainTurtle)

In [ ]:
sns.scatterplot(x="Capture_Number", y="Weight_Kg", data=trainTurtle)

In [ ]:
sns.boxplot(x="Date_TimeCaughtYear", y="Capture_Number", 
            data=trainTurtle[(trainTurtle["Date_TimeCaughtYear"] > 1997) & (trainTurtle["Date_TimeCaughtYear"] < 2008)])

In [ ]:
sns.boxplot(x="Date_TimeCaughtYear", y="Capture_Number", 
            data=trainTurtle[(trainTurtle["Date_TimeCaughtYear"] > 2007) & (trainTurtle["Date_TimeCaughtYear"] < 2019)])

In [ ]:
sns.lineplot(x="Date_TimeCaughtYear", y="Capture_Number", 
             data=trainTurtle.groupby(["Date_TimeCaughtYear"])["Capture_Number"].agg("sum").reset_index())

Between 1998 and 2018, there has been a positive trend. This simply means, the number of rescued sea turtles has been increasing across the decades.

In [ ]:
years = sorted(trainTurtle["Date_TimeCaughtYear"].unique().tolist())

for year in years[:5]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year, 
                 data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("sum").reset_index())

plt.tight_layout()

In [ ]:
for year in years[5:10]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("sum").reset_index())

plt.tight_layout()

In [ ]:
for year in years[10:15]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("sum").reset_index())

plt.tight_layout()

In [ ]:
for year in years[15:]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("sum").reset_index())

plt.tight_layout()

In [ ]:
sns.lineplot(x="Date_TimeCaughtYear", y="Capture_Number", 
             data=trainTurtle.groupby(["Date_TimeCaughtYear"])["Capture_Number"].agg("mean").reset_index())

In [ ]:
years = sorted(trainTurtle["Date_TimeCaughtYear"].unique().tolist())

for year in years[:5]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year, 
                 data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("mean").reset_index())

plt.tight_layout()

In [ ]:
for year in years[5:10]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("mean").reset_index())

plt.tight_layout()

In [ ]:
for year in years[10:15]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("mean").reset_index())

plt.tight_layout()

In [ ]:
for year in years[15:]:
    sns.lineplot(x="Date_TimeCaughtWeek", y="Capture_Number", label=year,
             data=trainTurtle[trainTurtle["Date_TimeCaughtYear"] == year].groupby(["Date_TimeCaughtWeek"])["Capture_Number"].agg("mean").reset_index())

plt.tight_layout()

In [ ]:
trainTurtle[["Date_TimeCaughtYear", "Capture_Number"]].head()

In [ ]:
trainTurtle["SMA_10"] = trainTurtle["Capture_Number"].rolling(5, min_periods=1).mean()

In [ ]:
trainTurtle["SMA_10"]